# LangChain 

https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/

https://python.langchain.com/v0.2/docs/tutorials/local_rag/

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [1]:
import os 

print(f"current working directory: {os.getcwd()}")

# os.chdir("")

current working directory: /Users/guchen/repo/LLM-complete/langchian


In [2]:
#### Indexing
from langchain_community.document_loaders import TextLoader

loader = TextLoader("/Users/guchen/repo/LLM-complete/data/txt/little_prince.txt")
docs = loader.load()


# split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = text_splitter.split_documents(docs)

# embed
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(chunks, OpenAIEmbeddings())

retriever = vector_store.as_retriever(search_kwargs={"k": 4})

print("index done")

index done


In [5]:
#
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

retrieve_template = """你需要根据用户的问题，从以下数据中选择最相关的数据，然后根据选择的数据生成一个回答。
问题：{question}
数据：{context}
回答："""

retrieve_template = ChatPromptTemplate.from_template(retrieve_template)
print(retrieve_template)

llm = ChatOpenAI(temperature=0, model="gpt-4o")

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = (
    {"content": retriever | format_docs, "question": RunnablePassthrough()}
    | retrieve_template
    | llm 
    | StrOutputParser()
)


input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你需要根据用户的问题，从以下数据中选择最相关的数据，然后根据选择的数据生成一个回答。\n问题：{question}\n数据：{context}\n回答：'))]


In [6]:
# 
response = rag_chain.invoke("这是一篇关于什么的小说？")


KeyError: "Input to ChatPromptTemplate is missing variables {'context'}.  Expected: ['context', 'question'] Received: ['content', 'question']"